## Question 2: Hamming’s Code (7,4)

*Write a simple Hamming encoder program in Python, which, when given a 4-bit binary value, returns the resulting 7-bit binary vector codeword. Also implement the parity check functionality to see if there are any errors, that is to check whether H ∗ cw = −→ 0 holds, where 0 is zero vector.*

In [ ]:
#Importing libraries needed for the project
import numpy as np
import random

## Implementation of Hamming (7,4) code and helper functions

In [ ]:
#Hammings generator matrix (G)
G = np.array(
    [[1, 1, 0, 1], 
	[1, 0, 1, 1],
	[1, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]])

#Parity-check matrix (H)
H = np.array(
    [[1, 0, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1]])

#Decoder Matrix (R)
R = np.array(
    [[0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1]])

#Function that checks if input is valid for the given function and converts it into list of integers
def validate_input(w, length): # O(n)
    # If w is an integer, convert it to a string
    if isinstance(w, int):
        w = str(w)
    # If w is a string, remove spaces and validate each character
    if isinstance(w, str):
        w = w.replace(" ", "")
        for i in w:
            if i not in ['0', '1']:
                raise Exception("Input is not binary")
        # Convert each element to an int and place in a list
        w = [int(i) for i in w]
    # Check if the length is correct
    if len(w) != length:
        raise Exception("Input does not have the correct length.")
    return w

#Function that flips a random number of bits to create a noisy message
def noisy_signal(message, num_flips): # O(n)
    noisy_message = [int(i) for i in message] # Convert message to list of integers
    flipped_bits = [] # List to store the flipped bit indices
    # Check if the number of flips is less than or equal to the message length
    if num_flips > len(noisy_message):
        raise ValueError("Number of flips cannot exceed the length of the message.")
    # Randomly choose bits to flip
    for _ in range(num_flips): # _ is a placeholder variable
        # Randomly select a bit index to flip
        bit_to_flip = random.randint(0, len(noisy_message) - 1)
        # Flip the bit (0 becomes 1, 1 becomes 0)
        noisy_message[bit_to_flip] = 1 if noisy_message[bit_to_flip] == 0 else 0
        # Add the flipped bit index to the list with 1-indexing
        flipped_bits.append(bit_to_flip + 1)
    if num_flips > 0:
        print("Oh no, a noisy signal has altered the message!")
        print("Flipped bit(s) at position(s):", flipped_bits)
        print("Noisy message:", np.array(noisy_message))
    return noisy_message

def hamming_encoder(w): # O(n)
    # Validate the input
    w = validate_input(w, 4)
    # Turn input string into NumPy array
    w = np.array(w)
    # Performs dot product multiplication and modulo operation to generate a new encoded binary message (codeword)
    cw = np.dot(G, w) % 2
    print("Encoded message: ", cw)
    return cw

# Function that performs parity check on the encoded message
def parity_check(cw): # O(n)
    cw = validate_input(cw, 7)
    s = np.dot(H, np.array(cw)) % 2
    print("Error syndrome vector:", s)
    return s

# Function that corrects the error in the encoded message
def error_correction(s, cw): # O(n)
    # Convert error syndrome to decimal to find error position
    error_pos = int(''.join(map(str, s[::-1])), 2)
    if error_pos:
        print(f"Error detected at position {error_pos}")
        # Correct the error
        cw[error_pos - 1] = 1 if cw[error_pos - 1] == 0 else 0
        print("Corrected encoded message:", cw)
    else:
        print("No error detected.")
    return cw

# Function that decodes the codeowrd to get the original message
def hamming_decoder(cw): # O(n)
    cw = validate_input(cw, 7)
    decoded_w = np.dot(R, cw) % 2 
    print("Decoded message:", decoded_w)
    return decoded_w

## Manual testing of Hamming (7,4) algorithm

In [ ]:
# Encoding and decoding without noise / error
cw = hamming_encoder("1001")
# Perform parity check
s = parity_check(cw)
# Correct any errors
cw_corrected = error_correction(s, cw)
# Decode the corrected message
decoded_w = hamming_decoder(cw_corrected)

In [ ]:
# Encode a message
cw = hamming_encoder("1001")
# Introduce a one-bit error
cw_noisy = noisy_signal(cw, 1)
# Perform parity check
s = parity_check(cw_noisy)
# Correct any errors
cw_corrected = error_correction(s, cw_noisy)
# Decode the corrected message
decoded_w = hamming_decoder(cw_corrected)

## Automated randomized testing of Hamming (7,4) algorithm

In [ ]:
def test_hamming_code(flips=0):
    # Ensure the number of flips is within the acceptable range
    if flips not in [0, 1, 2]:
        raise ValueError("Number of flips must be 0, 1, or 2")

    # Step 1: Generate random 4-bit input
    original_message = np.random.randint(0, 2, 4)
    print("Random 4-bit input:", original_message)

    # Step 2: Encode the input using hamming_encoder_84
    encoded_message = hamming_encoder(original_message)

    # Step 3: Apply noisy_signal to modify the encoded word
    noisy_message = noisy_signal(encoded_message, flips)

    # Step 4: Run a parity check
    error_syndrome = parity_check(noisy_message)

    # Ensure error_syndrome is not None before proceeding
    if error_syndrome is None:
        print("Error in computing the error syndrome.")
        return

    # Convert error syndrome to decimal to find the position of the error
    error_position = int(''.join(map(str, error_syndrome[::-1])), 2)
        #^Reverses the order of elements in the error syndrome vector for correct interpreation of binary mapping to decimal.
        #^In Hamming codes, the error syndrome is read from right to left when translating it into a decimal position.

    if error_position == 0:
        print("No errors detected.")
    elif error_position > 0:
        print("Single error detected at position", error_position)
        # Correct the single error by inverting specific bit (subtract 1 because arrays are 0-indexed)
        noisy_message[error_position - 1] = 1 if noisy_message[error_position - 1] == 0 else 0
        print("Corrected message:", noisy_message)
    else:
        print("Multiple errors detected. Cannot be recovered.")

    # Step 5: Decode the corrected message using the decoder matrix R
    decoded_message = hamming_decoder(noisy_message)

    # Step 6: Compare the decoded message with the original message
    if np.array_equal(original_message, decoded_message):
        print("Success: Original message successfully recovered.\n")
    else:
        print("Failure: Original message not recovered.\n")


# Run 3 tests
test_hamming_code(flips=0)
test_hamming_code(flips=1)
test_hamming_code(flips=2)